<a href="https://colab.research.google.com/github/alyssa-tsh/CS3244_ML_Project/blob/main/cs3244_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from functions import data_pipeline

/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train, test = data_pipeline()

Loading data


100%|██████████| 3.04M/3.04M [00:01<00:00, 2.48MB/s]

Extracting zip of application_record.csv...


100%|██████████| 2.28M/2.28M [00:01<00:00, 1.97MB/s]

Extracting zip of credit_record.csv...


Splitting data
Total unique accounts: 45985. Starting to find cutoff point
Cutoff month where CDF reaches 80%: -10

=== Split based on CDF 80% cutoff ===
Cutoff month: -10 (10 months ago)
Old accounts (≤ month -10): 37,210 (80.9%)
New accounts (> month -10): 8,775 (19.1%)
Ratio (old/new): 4.2405
Splitting raw credit records
Cleaning old accounts credit records - [Length: 996586]


/workspaces/CS3244_ML_Project/functions.py:174: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_df = df.groupby(['id', 'origination_month']).apply(lambda x: pd.Series({


Cleaning new accounts credit records - [Length: 51989]


/workspaces/CS3244_ML_Project/functions.py:174: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_df = df.groupby(['id', 'origination_month']).apply(lambda x: pd.Series({
/workspaces/CS3244_ML_Project/functions.py:81: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_dropped=df_sorted.groupby('id', group_keys=False).apply(keep_row)


Cleaning credit data completed
Splitting application dataset
Starting to clean application records
Dropping object columns
Completed cleaning raw application records
Merging data
Engineering target variable to label data
Completed old accounts labelling
Completed new accounts labelling
Old accounts: (37210, 3)
New accounts: (8775, 3)
Old threshold: 0.19907407407407407
New threshold: 0.5786182336182336
Merging cleaned application and credit records
Train shape: (29264, 28)
Test shape: (7193, 28)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29264 entries, 0 to 29263
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           29264 non-null  int64  
 1   cnt_children                 29264 non-null  int64  
 2   amt_income_total             29264 non-null  float64
 3   days_birth                   29264 non-null  int64  
 4   days_employed                29264 non

## Model Pipeline

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer 
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd

### Functions

In [ ]:
def get_n_components(X_train, method = "avg"):
        pca = PCA()
        pca.fit(X_train)
        explained_var_ratio = pca.explained_variance_ratio_
        if method == "avg":
            avg_var = 1 / len(explained_var_ratio)
            optimal_components = np.sum(explained_var_ratio > avg_var)

        elif method == "elbow":
            diffs = np.diff(explained_var_ratio)
            elbow_idx = np.argmax(diffs * -1) + 1 
            optimal_components = elbow_idx

        elif method == "cumulative":
            cum_var = np.cumsum(explained_var_ratio)
            optimal_components = np.argmax(cum_var >= 0.95) + 1
        else:
            # comparison with a base model to see if PCA methods are actually improving the model
            optimal_components = None
        return optimal_components
    
methods = ["avg", "elbow", "cumulative", "default"]

### Transformations & Scaling
- from QQ plot analysis - noticed that certain numerical features require transformation and diff features need different scalers
> Highly skewed & outlier-heavy	risk_score, months_employed	→ YeoJohnsonTransformer() or np.log1p() → RobustScaler

> Already normal/log-transformed	amt_income_total_log, age	→ StandardScaler

> Discrete / ordinal numeric	cnt_children, cnt_fam_members	→ Keep as is or encode as ordinal integers

### Encoding
* REALIZED that there are a lot of categorical features - label encoder might assign encoded categories some inherent ordering affecting model which is fine for tree based models but not for SVC and XGboost
| Feature type                  | XGBoost                     | SVC          | KNN                       |
| ----------------------------- | --------------------------- | ------------ | ------------------------- |
| Binary                        | 0/1 mapping                 | 0/1 mapping  | 0/1 mapping               |
| Low-cardinality (<5)          | One-hot or label encoding   | One-hot only | One-hot                   |
| Medium/high-cardinality (~17) | Frequency or label encoding | One-hot only | One-hot / binary encoding |
| Numeric                       | Raw                         | Standardized | Standardized              |


In [ ]:
def encode_features(df, target_col=None, encoders=None):
    if encoders is None:
        encoders = {}
    
    df_encoded = df.copy()
    
    # Define your columns
    binary_cols = ["code_gender", "flag_own_realty", "flag_own_car"]
    low_card_cols = ["name_income_type", "name_education_type", "name_family_status", "name_housing_type"]
    high_card_cols = ["occupation_type", "age_binned"]
    
    # ---- Binary mapping ----
    for col in binary_cols:
        mapping = {'M':0, 'F':1, 'Y':1, 'N':0}
        df_encoded[col+"_encoded"] = df_encoded[col].map(mapping)
    
    # ---- One-hot encoding for low-cardinality ----
    for col in low_card_cols:
        if col not in encoders:
            ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
            ohe.fit(df_encoded[[col]])
            encoders[col] = ohe
        else:
            ohe = encoders[col]
        
        cols_encoded = pd.DataFrame(ohe.transform(df_encoded[[col]]),
                                    columns=[f"{col}_{c}" for c in ohe.categories_[0]],
                                    index=df_encoded.index)
        df_encoded = pd.concat([df_encoded, cols_encoded], axis=1)
    
    # ---- Target encoding for high-cardinality ----
    for col in high_card_cols:
        if col not in encoders:
            if target_col is None:
                raise ValueError(f"Target column must be provided for training target encoding on {col}")
            te = TargetEncoder()
            te.fit(df_encoded[[col]], df_encoded[target_col])
            encoders[col] = te
        else:
            te = encoders[col]
        
        df_encoded[col+"_encoded"] = te.transform(df_encoded[[col]])
    
    # ---- Drop original object columns ----
    object_cols = df_encoded.select_dtypes('object').columns
    df_encoded.drop(columns=object_cols, inplace=True)
    
    return df_encoded, encoders

In [ ]:
def model_pipeline(train_df, test_df, target_col, numeric_cols, categorical_cols=None, random_state=42):

    # drop highly correlated features - keeping months employed 
    # assigning numeric_cols specific scalers and transformations based on QQ plot analysis
    skewed = ['risk_score', 'months_employed']
    normal = ['amt_income_total_log', 'age']
    discrete = ['cnt_children', 'cnt_fam_members']
    drop_cols = ["days_birth", "amt_income_total", "years_employed", "flag_mobil"]


    train_df = train_df.drop(columns=drop_cols)
    test_df = test_df.drop(columns=drop_cols)

    train = train_df.copy()
    test = test_df.copy()

    # train test split
    X_train_full = train.drop(columns=[target_col])
    y_train_full = train[target_col]
    X_test = test.drop(columns=[target_col])
    y_test = test[target_col]

    # Using RobustScaler instead since from EDA Standard & Min-Max Scaler distorted by outliers
    preprocessor = ColumnTransformer(
    transformers=[
        ('skewed', Pipeline([
            ('yeo', PowerTransformer(method='yeo-johnson')),
            ('robust', RobustScaler())
        ]), skewed),
        
        ('normal', StandardScaler(), normal),
        
        # No scaling required for discrete features
        ('discrete', 'passthrough', discrete)

    ]

    low_card_cols = ["name_income_type","name_education_type","name_family_status","name_housing_type","age_binned"]
    high_card_cols = ["occupation_type"]

    xgb_preprocessor = ColumnTransformer(
        transformers=[
            ("num", "passthrough", numeric_cols),
            ("low_cat", OneHotEncoder(handle_unknown='ignore'), low_card_cols),
            ("high_cat", FunctionTransformer(lambda X: X.assign(**{col: X[col].map(X[col].value_counts(normalize=True)) 
                                                                for col in high_card_cols})), high_card_cols)
        ]
    )

    xgb_pipeline = Pipeline([
        ("preprocessor", xgb_preprocessor),
        ("classifier", XGBClassifier())
    ])

    # Example for SVC
    svc_preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numeric_cols),
            ("cat", OneHotEncoder(handle_unknown='ignore'), low_card_cols + high_card_cols)
        ]
    )

    svc_pipeline = Pipeline([
        ("preprocessor", svc_preprocessor),
        ("classifier", SVC(probability=True))
    ])

    
)

    # Define models
    models = {
        "SVM (Linear)": SVC(kernel="linear", random_state=random_state),
        "XGB Classifier" : XGBClassifier(use_label_encoder=True, eval_metric="logloss", random_state=random_state)
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    results = []

    for name, model in models.items():
        print(f"Training model: {name} with StratifiedKFold...")
        acc_scores, f1_scores, roc_scores = [], [], []

        for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_full, y_train_full), 1):
            X_train, X_val = X_train_full.iloc[train_idx], X_train_full.iloc[val_idx]
            y_train, y_val = y_train_full.iloc[train_idx], y_train_full.iloc[val_idx]

            pipeline = Pipeline([
                ("preprocess", preprocessor),
                ("classifier", model)
            ])
            pipeline.fit(X_train, y_train)

            y_pred = pipeline.predict(X_val)
            y_proba = pipeline.predict_proba(X_val)[:,1] if hasattr(pipeline, "predict_proba") else None

            acc_scores.append(accuracy_score(y_val, y_pred))
            f1_scores.append(f1_score(y_val, y_pred))
            roc_scores.append(roc_auc_score(y_val, y_proba) if y_proba is not None else np.nan)

            print(f"Fold {fold}: Accuracy={acc_scores[-1]:.3f}, F1={f1_scores[-1]:.3f}, ROC-AUC={roc_scores[-1]:.3f}")

        results.append({
            "model": name,
            "accuracy": np.mean(acc_scores),
            "f1_score": np.mean(f1_scores),
            "roc_auc": np.nanmean(roc_scores)
        })
        print(f"Finished training {name} across all folds.\n")

    results_df = pd.DataFrame(results).sort_values(by="f1_score", ascending=False)

    return results_df, X_train_full, y_train_full, X_test, y_test


target_col = "label"
results_df, X_train, y_train, X_test, y_test = model_pipeline(
    train, test, target_col, numeric_cols
)